In [51]:
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3, VGG16

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

from os import listdir
from os.path import isdir, isfile, join

In [27]:
train_data_dir = r'D:\Praca\Data_Science\SDA_Kurs\Grapevine_leaves\Data\Train_val'
test_data_dir = r'D:\Praca\Data_Science\SDA_Kurs\Grapevine_leaves\Data\Test'

In [13]:
img_width = 300
img_height = 300

EPOCHS = 20
BATCH_SIZE = 32

IMAGE_SHAPE = (img_width, img_height)

In [18]:
datagen = ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.1 # set validation split
)

In [17]:
CLASSES = sorted(listdir(data_dir))
CLASSES

['Ak', 'Ala_Idris', 'Buzgulu', 'Dimnit', 'Nazli']

In [28]:
train_gen = datagen.flow_from_directory(
    train_data_dir, 
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES,
    subset='training',
    shuffle=True
)

validation_gen = datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes = CLASSES,
    subset='validation',
    shuffle=True
)

test_gen = datagen.flow_from_directory(
    test_data_dir,
    target_size=IMAGE_SHAPE,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

Found 405 images belonging to 5 classes.
Found 45 images belonging to 5 classes.
Found 50 images belonging to 1 classes.


# BASIC SEQUENTIAL MODEL

In [42]:
model = Sequential()

# Add a 2D convolutional layer with 32 filters of size 3x3 and ReLU activation function
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))

# Add a MaxPooling layer to reduce the data size
model.add(MaxPooling2D((2, 2)))

# Add another 2D convolutional layer with 64 filters of size 3x3 and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling layer
model.add(MaxPooling2D((2, 2)))

# Add a 2D convolutional layer with 128 filters of size 3x3 and ReLU activation
model.add(Conv2D(128, (3, 3), activation='relu'))

# Add another MaxPooling layer
model.add(MaxPooling2D((2, 2)))

# Flatten the data into a one-dimensional array
model.add(Flatten())

# Add a dense layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))

# Add an output layer with one neuron and a sigmoid activation function (binary classification)
model.add(Dense(len(CLASSES), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.0001), 
              metrics=['accuracy'])


In [43]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=EPOCHS,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)
)

# STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
# STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

# history = model.fit_generator(generator=train_gen,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=validation_gen,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=EPOCHS
# )

Epoch 1/20
13/13 [==============================] - 93s 7s/step - loss: 2.5140 - accuracy: 0.1926 - val_loss: 1.6024 - val_accuracy: 0.2667
Epoch 2/20
13/13 [==============================] - 85s 6s/step - loss: 1.6291 - accuracy: 0.2049 - val_loss: 1.6167 - val_accuracy: 0.2000
Epoch 3/20
13/13 [==============================] - 79s 6s/step - loss: 1.5501 - accuracy: 0.3037 - val_loss: 1.6001 - val_accuracy: 0.2222
Epoch 4/20
13/13 [==============================] - 82s 6s/step - loss: 1.4948 - accuracy: 0.3506 - val_loss: 1.6107 - val_accuracy: 0.2222
Epoch 5/20
13/13 [==============================] - 82s 6s/step - loss: 1.3841 - accuracy: 0.5062 - val_loss: 1.5744 - val_accuracy: 0.2444
Epoch 6/20
13/13 [==============================] - 94s 7s/step - loss: 1.2700 - accuracy: 0.5457 - val_loss: 1.5926 - val_accuracy: 0.2889
Epoch 7/20
13/13 [==============================] - 142s 11s/step - loss: 1.1125 - accuracy: 0.6272 - val_loss: 1.5770 - val_accuracy: 0.2222
Epoch 8/20
13/13 [

In [45]:
model.save('basic_model.keras')

# Model 1

### WITH DROPOUTS, BATCH_NORM AND EARLY STOPPING

In [52]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(CLASSES), activation='softmax'))

In [53]:
# Compile the model
opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy'])

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

history = model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=EPOCHS,
                    callbacks=[callback],
                    verbose=1
)

C:\Users\Katarzyna\AppData\Local\Temp\ipykernel_2984\2730287419.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_gen,


Epoch 1/20
12/12 [==============================] - 114s 9s/step - loss: 2.2249 - accuracy: 0.2895 - val_loss: 15.8087 - val_accuracy: 0.2188
Epoch 2/20
12/12 [==============================] - 102s 8s/step - loss: 0.9912 - accuracy: 0.6193 - val_loss: 36.8633 - val_accuracy: 0.1875
Epoch 3/20
12/12 [==============================] - 115s 10s/step - loss: 0.7851 - accuracy: 0.7109 - val_loss: 26.3906 - val_accuracy: 0.1562
Epoch 4/20
12/12 [==============================] - 130s 11s/step - loss: 0.5943 - accuracy: 0.8043 - val_loss: 29.2213 - val_accuracy: 0.2812
Epoch 5/20
12/12 [==============================] - 152s 13s/step - loss: 0.3909 - accuracy: 0.8794 - val_loss: 22.2857 - val_accuracy: 0.2188
Epoch 6/20
12/12 [==============================] - 145s 12s/step - loss: 0.2531 - accuracy: 0.9115 - val_loss: 16.4876 - val_accuracy: 0.3125
Epoch 7/20
12/12 [==============================] - 134s 11s/step - loss: 0.1181 - accuracy: 0.9740 - val_loss: 14.9828 - val_accuracy: 0.1875
E

In [54]:
model.save('model1.keras')

In [55]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(CLASSES), activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

# Model 2

In [56]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=EPOCHS,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)
)

Epoch 1/20
13/13 [==============================] - 261s 19s/step - loss: 2.0993 - accuracy: 0.3704 - val_loss: 1.7191 - val_accuracy: 0.2444
Epoch 2/20
13/13 [==============================] - 228s 17s/step - loss: 0.6250 - accuracy: 0.7951 - val_loss: 2.3985 - val_accuracy: 0.2000
Epoch 3/20
13/13 [==============================] - 199s 15s/step - loss: 0.2955 - accuracy: 0.9210 - val_loss: 3.5069 - val_accuracy: 0.2000
Epoch 4/20
13/13 [==============================] - 190s 15s/step - loss: 0.1333 - accuracy: 0.9877 - val_loss: 4.5011 - val_accuracy: 0.2000
Epoch 5/20
13/13 [==============================] - 188s 14s/step - loss: 0.0619 - accuracy: 0.9926 - val_loss: 4.9750 - val_accuracy: 0.2000
Epoch 6/20
13/13 [==============================] - 204s 16s/step - loss: 0.0322 - accuracy: 0.9975 - val_loss: 4.8684 - val_accuracy: 0.2000
Epoch 7/20
13/13 [==============================] - 207s 15s/step - loss: 0.0198 - accuracy: 0.9975 - val_loss: 5.0436 - val_accuracy: 0.1556
Epoch 

KeyboardInterrupt: 